# Subprocess

Assume we want to: send and Receive Data from a process

1. Create a new process that launches the command line argument
2. Send the string Testing\n to that process (should use pipe)
3. Receive any data that this new process writes to standard output $\implies$ use pipe to set this process's stdout to write to pipe 

## System call – `execlp`

Recall: we can use `execve` to replace a process with a different process (restarted), but this is kinda inconvenient

    int execlp(const char *file, const char *arg /*..., (char *) NULL ⁎/);

- Only need to specify a file name (doesn't need to specify the whole path) - will look for default executables in <mark>**PATH Environment variables**</mark>
- (similar to `execve`) Does not return on success (since we're already in a different process), and -1 on failure (and sets errno)  

`execlp` will let you skip using string arrays (using C varargs), and it will also search for executables using the PATH environment variable

- <mark>NOTE: This will just simply loads the new program's instructions into the memory, and execute that, but **doesn't close the file descriptors**</mark>
- <mark>The pid will stay the same (and the same parent as well)</mark>

## System call — `dup` and `dup2`

    int dup(int oldfd);
    int dup2(int oldfd, int newfd);

- Returns a new file descriptor on success, and -1 on failure (and sets errno)

- Copies the file descriptor so **oldfd and newfd refer to the same thing**

For `dup` it’ll return the lowest file descriptor, which will refer to oldfd

For `dup2` it’ll **automically close the newfd argument (if open)**, and then make newfd refer to oldfd

***
## Coding Example

In [48]:
%%file run.c

#include <assert.h>
#include <errno.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/wait.h>
#include <unistd.h>

static void check_error(int ret, const char *message) {
    if (ret != -1) {
        return;
    }
    int err = errno;
    perror(message);
    exit(err);
}

static void parent(int in_pipefd[2], int out_pipefd[2], pid_t child_pid) {
    close(in_pipefd[0]);
    close(out_pipefd[1]);
    
    const char* message = "Testing\n";
    ssize_t bytes_written = write(in_pipefd[1], message, strlen(message));
    check_error(bytes_written, "write");
    close(in_pipefd[1]);
    
    int wstatus;
    check_error(wait(&wstatus), "wait");
    assert(WIFEXITED(wstatus));
    assert(WEXITSTATUS(wstatus) == 0);
    
    char buffer[4096];
    ssize_t bytes_read = read(out_pipefd[0], buffer, sizeof(buffer));    // Recall: read is blocking
    check_error(bytes_read, "read");
    printf("Got: %s\n", buffer);
    close(out_pipefd[0]);
}

static void child(int in_pipefd[2], int out_pipefd[2], const char *program) {
    ////////////////////////////////////
    dup2(in_pipefd[0], 0);
    dup2(out_pipefd[1], 1); 
    ////////////////////////////////////
    
    close(in_pipefd[0]);
    close(in_pipefd[1]);
    close(out_pipefd[0]);
    close(out_pipefd[1]);   // Closed out_pipefd[1], but didn't close fd = 1 (which is also referring to the write end of the pipe by now)
    
    execlp(program, program, NULL);
}

int main(int argc, char* argv[]) {
    if (argc != 2) {
        return EINVAL;
    }

    int in_pipefd[2] = {0};
    check_error(pipe(in_pipefd), "pipe");

    int out_pipefd[2] = {0};
    check_error(pipe(out_pipefd), "pipe");

    pid_t pid = fork();
    if (pid > 0) {
        parent(in_pipefd, out_pipefd, pid);
    }
    else {
        child(in_pipefd, out_pipefd, argv[1]);
    }

    return 0;
}


Overwriting run.c


In [49]:
!gcc run.c -g -o run.o

In [51]:
!./run.o uname

Got: Darwin



If Parent runs first:

- Close **in_pipe read**, **out_pipe write** (on parent side)
- It writes "Testing" to **in_pipe write**
- Close **in_pipe write** (on parent side)
- Try to read from **out_pipe read**. This will be blocking -> sleeps

- Child process runs.
- Changes fd = 0 to **in_pipe read**
- Changes fd = 1 to **out_pipe write**
- Close everything on child side
- Switch process. 
    - `uname` simply outputs to fd = 1 $\implies$ outputs to **out_pipe write**
    - `cat` takes things from fd = 0, output to fd = 1
        - $\implies$ Took "Testing" from **in_pipe** and output to **out_pipe**

- Parent continues
- Print **out_pipe read**
- Close **out_pipe read** (on parent side)

If Child runs first:

- Changes fd = 0 to **in_pipe read**
- Changes fd = 1 to **out_pipe write**
- Close everything on child side
- Switch process. 
    - `uname` simply outputs to fd = 1 $\implies$ outputs to **out_pipe write**
    - `cat` takes things from fd = 0, output to fd = 1 $\implies$ There is nothing in **in_pipe** yet, sleep

- Parent process runs
- It writes "Testing" to **in_pipe write**
- Try to read from **out_pipe read** $\implies$ Nothing here yet, sleep

- Child continues.
- `cat` reads "Testing", output to **out_pipe**

- Parent continues
- Print